In [1]:
import pandas as pd
import numpy as np

# Read in and filter on years

In [2]:
df = pd.read_csv('cumulative_2006_2017.csv')
df = df[np.logical_or(np.logical_or(df['year'] == 2006,df['year'] == 2008),df['year'] == 2010)]
print(np.shape(df))

C:\Users\jerem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(124621, 70)


# One-Hot and add, simple

In [3]:
dummy = pd.get_dummies(df['gender'])
df = df.drop('gender',axis=1)
dummy.columns = ['male','female']
df = df.join(dummy)
dummy = pd.get_dummies(df['race'])
df = df.drop('race',axis=1)
raceList = ['White','Black','Hispanic','Asian','Native American','Mixed','Other','Middle Eastern']
dummy.columns= raceList
df = df.join(dummy)
#assume floating point error correspond to no's, this is supported by the distribution of hispanic/not hispanic in the CCES guide
df['hispanic'][df['hispanic'] == -2147483648] = 2
dummy = pd.get_dummies(df['hispanic'])
dummy.columns = ['Yes Hispanic','No Hispanic']
df = df.drop('hispanic',axis=1)
df = df.join(dummy)

df = df[df['state'] != 'District of Columbia']
stateList = df['state'].value_counts().keys().tolist()
yearList = [2006,2008,2010]
dfNew = pd.DataFrame(columns=df.columns)
stateCounter = 0
for i in range(50*3):
    dfNew = dfNew.append(pd.Series([np.nan]),ignore_index=True)
    dfNew['state'].iloc[i] = stateList[stateCounter]
    dfNew['year'].iloc[i] = yearList[i%3]
    if (i) % 3 == 2:
        stateCounter += 1

        
featList = ['White','Black','Hispanic','Asian','Native American','Mixed','Other','Middle Eastern','male','female','Yes Hispanic','No Hispanic']
for i, row in dfNew.iterrows():
    print(i)
    temp = df[np.logical_and(df['state'] == row['state'],df['year'] == row['year'])]
    for s in featList:
        dfNew[s].iloc[i] = temp[s].sum()


C:\Users\jerem\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\jerem\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [4]:
featList = ['year','state','White','Black','Hispanic','Asian','Native American','Mixed','Other','Middle Eastern','male','female','Yes Hispanic','No Hispanic']

In [5]:
dfNew = dfNew[featList]
for i, row in dfNew.iterrows():
    racesum = row[raceList].sum()
    for s in raceList:
        dfNew[s].iloc[i] = float(dfNew[s].iloc[i])/float(racesum)
    gendersum = row['male'] + row['female']
    dfNew['male'].iloc[i] = float(row['male'])/float(gendersum)
    dfNew['female'].iloc[i] = float(row['female'])/float(gendersum)

C:\Users\jerem\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
dfNew

,year,state,White,Black,Hispanic,Asian,Native American,Mixed,Other,Middle Eastern,male,female,Yes Hispanic,No Hispanic
0,2006.0,California,0.615342,0.094858,0.203464,0.019247,0.007424,0.026945,0.029695,0.003024,0.504812,0.495188,0,3637
1,2008.0,California,0.693087,0.035964,0.167947,0.041201,0.009078,0.025140,0.026187,0.001397,0.513966,0.486034,0,2864
2,2010.0,California,0.601443,0.106106,0.179307,0.033433,0.009150,0.032729,0.035545,0.002288,0.493753,0.506247,355,5328
3,2006.0,Florida,0.714653,0.070694,0.182948,0.002571,0.003856,0.007284,0.015853,0.002142,0.508997,0.491003,0,2334
4,2008.0,Florida,0.765325,0.064501,0.124886,0.007319,0.006404,0.011894,0.018756,0.000915,0.465233,0.534767,0,2186
5,2010.0,Florida,0.719595,0.099626,0.130416,0.007038,0.006598,0.013415,0.022652,0.000660,0.477238,0.522762,169,4378
6,2006.0,Texas,0.638092,0.095409,0.225610,0.003945,0.008608,0.007532,0.018293,0.002511,0.515423,0.484577,0,2788
7,2008.0,Texas,0.715834,0.074458,0.155985,0.013195,0.010839,0.012724,0.016023,0.000943,0.455231,0.544769,0,2122
8,2010.0,Texas,0.648474,0.123642,0.158303,0.011381,0.008019,0.017072,0.031816,0.001293,0.481117,0.518883,120,3746
9,2006.0,New York,0.727068,0.120990,0.109736,0.006753,0.003939,0.011255,0.015757,0.004502,0.492966,0.507034,0,1777


In [14]:
txt = open('finallyProcessed2.csv').readlines()
proc = pd.DataFrame(columns=[i for i in range(len(txt))])
for i, ln in enumerate(txt):
    row_items = ln.split(',')
    proc[i] = row_items

In [20]:
for i, row in proc.iterrows():
    if len(row.value_counts()) > 1:
        print(row[0])
        print(row.value_counts().keys())


Index(['207', '948', '497', '671', '1603', '885', '930', '1534', '391', '2043',
       ...
       '2060', '1671', '1837', '41', '1684', '192', '1087', '1718', '151', ''],
      dtype='object', length=2190)
Unnamed: 0
Index(['207', '1534', '533', '497', '671', '1603', '885', '930', '948', '1229',
       ...
       '2060', '1671', '1837', '41', '1684', '192', '1087', '1718', '151',
       '1283'],
      dtype='object', length=2190)
Unnamed: 0.1
Index(['207', '1534', '533', '497', '671', '1603', '885', '930', '948', '1229',
       ...
       '1671', '1837', '41', '1684', '192', '1087', '1718', '151', '108',
       '1283'],
      dtype='object', length=2190)
CNUM
Index(['111.0', '110.0', '113.0', '112.0', 'CNUM'], dtype='object')
ICPSR
Index(['99911.0', '20115.0', '29534.0', '20730.0', '14435.0', '20304.0',
       '20331.0', '20305.0', '14854.0', '15431.0',
       ...
       '21319.0', '20936.0', '29136.0', '21356.0', '20943.0', '29100.0',
       '20141.0', '29355.0', '21339.0', '21104.0'

117721
Index(['0.0', '117721'], dtype='object')
118828
Index(['0.0', '118828'], dtype='object')
119040
Index(['0.0', '119040'], dtype='object')
119354
Index(['0.0', '119354'], dtype='object')
119388
Index(['0.0', '119388'], dtype='object')
119701
Index(['0.0', '119701'], dtype='object')
120055
Index(['0.0', '120055'], dtype='object')
120683
Index(['0.0', '120683'], dtype='object')
1214
Index(['0.0', '1214'], dtype='object')
122101
Index(['0.0', '122101'], dtype='object')
122176
Index(['0.0', '122176'], dtype='object')
1230822927
Index(['0.0', '1230822927'], dtype='object')
123281
Index(['0.0', '123281'], dtype='object')
123372
Index(['0.0', '123372'], dtype='object')
123513
Index(['0.0', '123513'], dtype='object')
12401
Index(['0.0', '12401'], dtype='object')
12476
Index(['0.0', '12476'], dtype='object')
12518
Index(['0.0', '12518'], dtype='object')
125534
Index(['0.0', '125534'], dtype='object')
126086
Index(['0.0', '126086'], dtype='object')
126961
Index(['0.0', '126961'], dtype='obj

Index(['0.0', '164392'], dtype='object')
164418
Index(['0.0', '164418'], dtype='object')
164509
Index(['0.0', '164509'], dtype='object')
1648155659
Index(['0.0', '1648155659'], dtype='object')
164848
Index(['0.0', '164848'], dtype='object')
165159
Index(['0.0', '165159'], dtype='object')
165241
Index(['0.0', '165241'], dtype='object')
165324
Index(['0.0', '165324'], dtype='object')
165449
Index(['0.0', '165449'], dtype='object')
165548
Index(['0.0', '165548'], dtype='object')
165688
Index(['0.0', '165688'], dtype='object')
165852
Index(['0.0', '165852'], dtype='object')
165944
Index(['0.0', '165944'], dtype='object')
166017
Index(['0.0', '166017'], dtype='object')
166041
Index(['0.0', '166041'], dtype='object')
166272
Index(['0.0', '166272'], dtype='object')
166355
Index(['0.0', '166355'], dtype='object')
166504
Index(['0.0', '166504'], dtype='object')
166694
Index(['0.0', '166694'], dtype='object')
166710
Index(['0.0', '166710'], dtype='object')
166827
Index(['0.0', '166827'], dtype='

Index(['0.0', '-2147483648.54'], dtype='object')
-2147483648.55
Index(['0.0', '-2147483648.55'], dtype='object')
-2147483648.56
Index(['0.0', '-2147483648.56'], dtype='object')
-2147483648.57
Index(['0.0', '-2147483648.57'], dtype='object')
-2147483648.58
Index(['0.0', '-2147483648.58'], dtype='object')
-2147483648.59
Index(['0.0', '-2147483648.59'], dtype='object')
-2147483648.60
Index(['0.0', '-2147483648.60'], dtype='object')
-2147483648.61
Index(['0.0', '-2147483648.61'], dtype='object')
-2147483648.62
Index(['0.0', '-2147483648.62'], dtype='object')
-2147483648.63
Index(['0.0', '-2147483648.63'], dtype='object')
-2147483648.64
Index(['0.0', '-2147483648.64'], dtype='object')
-2147483648.65
Index(['0.0', '-2147483648.65'], dtype='object')
-2147483648.66
Index(['0.0', '-2147483648.66'], dtype='object')
-2147483648.67
Index(['0.0', '-2147483648.67'], dtype='object')
-2147483648.68
Index(['0.0', '-2147483648.68'], dtype='object')
-2147483648.69
Index(['0.0', '-2147483648.69'], dtype='

-2147483648.222
Index(['0.0', '-2147483648.222'], dtype='object')
-2147483648.223
Index(['0.0', '-2147483648.223'], dtype='object')
-2147483648.224
Index(['0.0', '-2147483648.224'], dtype='object')
-2147483648.225
Index(['0.0', '-2147483648.225'], dtype='object')
-2147483648.226
Index(['0.0', '-2147483648.226'], dtype='object')
-2147483648.227
Index(['0.0', '-2147483648.227'], dtype='object')
-2147483648.228
Index(['0.0', '-2147483648.228'], dtype='object')
-2147483648.229
Index(['0.0', '-2147483648.229'], dtype='object')
-2147483648.230
Index(['0.0', '-2147483648.230'], dtype='object')
-2147483648.231
Index(['0.0', '-2147483648.231'], dtype='object')
-2147483648.232
Index(['0.0', '-2147483648.232'], dtype='object')
-2147483648.233
Index(['0.0', '-2147483648.233'], dtype='object')
-2147483648.234
Index(['0.0', '-2147483648.234'], dtype='object')
-2147483648.235
Index(['0.0', '-2147483648.235'], dtype='object')
-2147483648.236
Index(['0.0', '-2147483648.236'], dtype='object')
-214748364

...

Index(['0.0', '-2147483648.1757'], dtype='object')
-2147483648.1758
Index(['0.0', '-2147483648.1758'], dtype='object')
-2147483648.1759
Index(['0.0', '-2147483648.1759'], dtype='object')
-2147483648.1760
Index(['0.0', '-2147483648.1760'], dtype='object')
-2147483648.1761
Index(['0.0', '-2147483648.1761'], dtype='object')
-2147483648.1762
Index(['0.0', '-2147483648.1762'], dtype='object')
-2147483648.1763
Index(['0.0', '-2147483648.1763'], dtype='object')
-2147483648.1764
Index(['0.0', '-2147483648.1764'], dtype='object')
-2147483648.1765
Index(['0.0', '-2147483648.1765'], dtype='object')
-2147483648.1766
Index(['0.0', '-2147483648.1766'], dtype='object')
-2147483648.1767
Index(['0.0', '-2147483648.1767'], dtype='object')
-2147483648.1768
Index(['0.0', '-2147483648.1768'], dtype='object')
-2147483648.1769
Index(['0.0', '-2147483648.1769'], dtype='object')
-2147483648.1770
Index(['0.0', '-2147483648.1770'], dtype='object')
-2147483648.1771
Index(['0.0', '-2147483648.1771'], dtype='object

Index(['0.0', '-2147483648.1892'], dtype='object')
-2147483648.1893
Index(['0.0', '-2147483648.1893'], dtype='object')
-2147483648.1894
Index(['0.0', '-2147483648.1894'], dtype='object')
-2147483648.1895
Index(['0.0', '-2147483648.1895'], dtype='object')
-2147483648.1896
Index(['0.0', '-2147483648.1896'], dtype='object')
-2147483648.1897
Index(['0.0', '-2147483648.1897'], dtype='object')
-2147483648.1898
Index(['0.0', '-2147483648.1898'], dtype='object')
-2147483648.1899
Index(['0.0', '-2147483648.1899'], dtype='object')
-2147483648.1900
Index(['0.0', '-2147483648.1900'], dtype='object')
-2147483648.1901
Index(['0.0', '-2147483648.1901'], dtype='object')
-2147483648.1902
Index(['0.0', '-2147483648.1902'], dtype='object')
-2147483648.1903
Index(['0.0', '-2147483648.1903'], dtype='object')
-2147483648.1904
Index(['0.0', '-2147483648.1904'], dtype='object')
-2147483648.1905
Index(['0.0', '-2147483648.1905'], dtype='object')
-2147483648.1906
Index(['0.0', '-2147483648.1906'], dtype='object

Index(['0.0', '-2147483648.2061'], dtype='object')
-2147483648.2062
Index(['0.0', '-2147483648.2062'], dtype='object')
-2147483648.2063
Index(['0.0', '-2147483648.2063'], dtype='object')
-2147483648.2064
Index(['0.0', '-2147483648.2064'], dtype='object')
-2147483648.2065
Index(['0.0', '-2147483648.2065'], dtype='object')
-2147483648.2066
Index(['0.0', '-2147483648.2066'], dtype='object')
-2147483648.2067
Index(['0.0', '-2147483648.2067'], dtype='object')
-2147483648.2068
Index(['0.0', '-2147483648.2068'], dtype='object')
-2147483648.2069
Index(['0.0', '-2147483648.2069'], dtype='object')
-2147483648.2070
Index(['0.0', '-2147483648.2070'], dtype='object')
-2147483648.2071
Index(['0.0', '-2147483648.2071'], dtype='object')
-2147483648.2072
Index(['0.0', '-2147483648.2072'], dtype='object')
-2147483648.2073
Index(['0.0', '-2147483648.2073'], dtype='object')
-2147483648.2074
Index(['0.0', '-2147483648.2074'], dtype='object')
-2147483648.2075
Index(['0.0', '-2147483648.2075'], dtype='object

-2147483648.2213
Index(['0.0', '-2147483648.2213'], dtype='object')
-2147483648.2214
Index(['0.0', '-2147483648.2214'], dtype='object')
-2147483648.2215
Index(['0.0', '-2147483648.2215'], dtype='object')
-2147483648.2216
Index(['0.0', '-2147483648.2216'], dtype='object')
-2147483648.2217
Index(['0.0', '-2147483648.2217'], dtype='object')
-2147483648.2218
Index(['0.0', '-2147483648.2218'], dtype='object')
-2147483648.2219
Index(['0.0', '-2147483648.2219'], dtype='object')
-2147483648.2220
Index(['0.0', '-2147483648.2220'], dtype='object')
-2147483648.2221
Index(['0.0', '-2147483648.2221'], dtype='object')
-2147483648.2222
Index(['0.0', '-2147483648.2222'], dtype='object')
-2147483648.2223
Index(['0.0', '-2147483648.2223'], dtype='object')
-2147483648.2224
Index(['0.0', '-2147483648.2224'], dtype='object')
-2147483648.2225
Index(['0.0', '-2147483648.2225'], dtype='object')
-2147483648.2226
Index(['0.0', '-2147483648.2226'], dtype='object')
-2147483648.2227
Index(['0.0', '-2147483648.2227

KeyboardInterrupt: 

In [10]:
len(row_items)

1

In [13]:
ln.split(',')

['0',
 '0',
 '0',
 '110.0',
 '99910.0',
 '99.0',
 '0.0',
 'USA',
 '200.0',
 'BUSH',
 '0.489',
 '-0.872',
 '0.1081',
 '0.135',
 '-0.068',
 '-31.916690000000006',
 '121.0',
 '12.0',
 '0.768',
 '3334',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 